In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle, faiss, os, sys
import pandas as pd
from collections import Counter
from scipy.stats import multivariate_normal

## 1. k Nearest Neighbours

### Auxiliary Functions 

In [ ]:
def euclidean_distance(data_df, point_1, point_2):
    
    array_1 = np.array(data_df.iloc[point_1, :-1])
    array_2 = np.array(data_df.iloc[point_2, :-1])
    dist = np.linalg.norm(array_1 - array_2)
    
    return [dist, data_df.iloc[point_2, -1]]

In [ ]:
def kNN(data_df, point, k):
    dist_list = [euclidean_distance(data_df, point, i) for i in range(len(data_df.index))]
    dist_list.sort(key=lambda x: x[0])
    
    return dist_list[1:k+1]

In [ ]:
def classify_kNN(data_df, point, k):
    
    kNN_list = kNN(data_df, point, k)
    freqDict = Counter([item[1] for item in kNN_list])
    size = len(kNN_list)
    
    for (key, val) in freqDict.items():
        if (val > (size/2)):
            return key

In [ ]:
def kNN_cross_validation(data_df, k):
    counter = 0
    for row in data_df.index.values:
        if classify_kNN(data_df, row, k) != data_df.iloc[row, -1]:
            counter += 1
    error_percentage = 100*counter/len(data_df.index.values)
    print(f"{k} NN - Misclassification Percentage: {error_percentage:.2f}%")
    
    return error_percentage

### Iris Plant Database

In [ ]:
def load_iris_data(data_path):
    
    header_list = [f"par_{i}" for i in range(1, 5)] + ["class"]
    data = pd.read_csv(data_path, sep=',', names=header_list, header=None)
    
    return data

In [ ]:
iris_df = load_iris_data(data_path = os.path.abspath(os.path.join(os.getcwd(), "UCIdata-exercise1", "iris.data")))

In [ ]:
iris_classes = list(set(iris_df.iloc[:, -1].values))
iris_class_values = dict([(key, [np.array(value[:-1]) for value in iris_df.values if value[-1]==key]) for key in iris_classes])

In [ ]:
print([kNN_cross_validation(iris_df, k) for k in range(1, 6)])

### Pima Indians Diabetes Database

In [ ]:
def load_pima_data(data_path):
    
    header_list = [f"par_{i}" for i in range(1, 9)] + ["class"]
    data = pd.read_csv(data_path, sep=',', names=header_list, header=None)
    
    return data

In [ ]:
pima_df = load_pima_data(data_path = os.path.abspath(os.path.join(os.getcwd(), "UCIdata-exercise1", "pima-indians-diabetes.data")))

In [ ]:
print(kNN_cross_validation(pima_df, 3))

## 2. PDF estimation

In [ ]:
N = len(pima_df.index.values)
k = len(pima_df.columns[:-1].values)

In [ ]:
pima_classes = list(set(pima_df.iloc[:, -1].values))
pima_class_values = dict([(key, [np.array(value[:-1]) for value in pima_df.values if value[-1]==key]) for key in pima_classes])

### Maximum Likelihood - Diagonal Covariance Matrix - Common Variance

In [ ]:
def ml_1(data_df):
    
    n = len(data_df.index.values)

    mu_ml_1 = (1/n)*(data_df.sum().values)
    x_minus_mu = data_df.sub(mu_ml_1)
    covar = np.average(x_minus_mu.pow(2).sum(axis=1))
    covar_1 = covar*np.identity(k)
    
    return mu_ml_1, covar_1

In [ ]:
mean_0_1, cov_0_1 = ml_1(pd.DataFrame(pima_class_values[0]))
mean_1_1, cov_1_1 = ml_1(pd.DataFrame(pima_class_values[1]))

In [ ]:
print(f"Class 0: μ={mean_0_1}")
print(f"Class 1: μ={mean_1_1}")

### Maximum Likelihood - Non-Diagonal Covariance Matrix

In [ ]:
def ml_2(data_df):
    
    n = len(data_df.index.values)
    
    mu_ml_2 = np.array((1/n)*data_df.sum())
    x_minus_mu = data_df.sub(mu_ml_2)
    
    inner_prod = 0
    
    for j in range(n):
        inner_prod += (np.array(x_minus_mu.loc[j]).reshape(k, 1))@(np.array(x_minus_mu.loc[j]).reshape(k, 1).transpose())
    
    covar_2 = (1/n)*inner_prod
    return mu_ml_2, covar_2

In [ ]:
mean_0_2, cov_0_2 = ml_2(pd.DataFrame(pima_class_values[0]))
mean_1_2, cov_1_2 = ml_2(pd.DataFrame(pima_class_values[1]))

### Maximum Likelihood - Statistically Independent Features

In [ ]:
def ml_3(data_df):
    
    n = len(data_df.index.values)

    mu_ml_3 = np.array((1/n)*data_df.sum())
    x_minus_mu = data_df.sub(mu_ml_3)

    diag_covar = (1/n)*(x_minus_mu.pow(2).sum())
    covar_3 = np.diag(diag_covar)
    
    return mu_ml_3, covar_3

In [ ]:
mean_0_3, cov_0_3 = ml_3(pd.DataFrame(pima_class_values[0]))
mean_1_3, cov_1_3 = ml_3(pd.DataFrame(pima_class_values[1]))

### Parzen Windows - Statistically Independent Features

In [ ]:
def parzen_1d(h, data, x):
    gaussian_results = [(1/(np.sqrt(2*np.pi)*h))*np.exp(-((x-x_i)**2)/(2*(h**2))) for x_i in data]
    return np.average(gaussian_results)

In [ ]:
def parzen_multivariate(h, data, x):
    gaussian_results = [(1/(np.sqrt(2*np.pi)*h))*np.exp(-(((x-x_i).transpose())@(x-x_i))/(2*(h**2))) for x_i in data]
    return np.average(gaussian_results)

In [ ]:
print(parzen_1d(np.sqrt(N), pd.DataFrame(pima_class_values[0]).iloc[:, :-1], pd.DataFrame(pima_class_values[1]).iloc[0, :-1]))

## 3. Bayesian Classification

In [ ]:
def gauss_prob(mu, cov, x):
    result = (1/(((2*np.pi)**(k/2))*(np.linalg.det(cov)**0.5)))*np.exp(-0.5*(((x-mu).transpose())@(np.linalg.pinv(cov))@(x-mu)))
    return result

### 3a. Maximum Likelihood - Σ = σI

In [ ]:
def classify_pima_1():
    
    class_df = pd.DataFrame(columns=["Assigned Class", "True Class"])
        
    for i in pima_df.index.values:
        temp_df = pima_df.drop(i)            
        temp_samples = dict([(key, [np.array(value[:-1]) for value in temp_df.values if value[-1]==key]) for key in pima_classes])
        
        mu_ml_0, covar_0 = ml_1(pd.DataFrame(temp_samples[0]))
        mu_ml_1, covar_1 = ml_1(pd.DataFrame(temp_samples[1]))
        
        x = np.array(pima_df.iloc[i, :-1])
        prob_0 = multivariate_normal.pdf(x=x, mean=mu_ml_0, cov=covar_0)
        prob_1 = multivariate_normal.pdf(x=x, mean=mu_ml_1, cov=covar_1)
        prob_class_0 = len(temp_samples[0])/(len(temp_samples[0]) + len(temp_samples[1]))
        prob_class_1 = len(temp_samples[1])/(len(temp_samples[0]) + len(temp_samples[1]))
        
        
        if prob_0*prob_class_0 > prob_1*prob_class_1:
            class_df.loc[i] = [0, pima_df.iloc[i, -1]]
        elif prob_1*prob_class_1 > prob_0*prob_class_0:
            class_df.loc[i] = [1, pima_df.iloc[i, -1]]
        else:
            class_df.loc[i] = ["NA", pima_df.iloc[i, -1]]
        
    return class_df

In [ ]:
class_df_1 = classify_pima_1()

In [ ]:
wrong_class_1 = abs(class_df_1["Assigned Class"] - class_df_1["True Class"]).sum()/len(pima_df.index.values)
print(f"% of wrong classification: {100*wrong_class_1:.0f}%")

### 3b. Maximum Likelihood - Non-Diagonal Σ

In [ ]:
def classify_pima_2():
    
    class_df = pd.DataFrame(columns=["Assigned Class", "True Class"])
        
    for i in pima_df.index.values:
        temp_df = pima_df.drop(i)            
        temp_samples = dict([(key, [np.array(value[:-1]) for value in temp_df.values if value[-1]==key]) for key in pima_classes])
                
        mu_ml_0, covar_0 = ml_2(pd.DataFrame(temp_samples[0]))
        mu_ml_1, covar_1 = ml_2(pd.DataFrame(temp_samples[1]))
        
        x = np.array(pima_df.iloc[i, :-1])
        prob_0 = multivariate_normal.pdf(x=x, mean=mu_ml_0, cov=covar_0)
        prob_1 = multivariate_normal.pdf(x=x, mean=mu_ml_1, cov=covar_1)
        prob_class_0 = len(temp_samples[0])/(len(temp_samples[0]) + len(temp_samples[1]))
        prob_class_1 = len(temp_samples[1])/(len(temp_samples[0]) + len(temp_samples[1]))

        
        if prob_0*prob_class_0 > prob_1*prob_class_1:
            class_df.loc[i] = [0, pima_df.iloc[i, -1]]
        elif prob_1*prob_class_1 > prob_0*prob_class_0:
            class_df.loc[i] = [1, pima_df.iloc[i, -1]]
        else:
            class_df.loc[i] = ["NA", pima_df.iloc[i, -1]]
        
    return class_df

In [ ]:
class_df_2 = classify_pima_2()

In [ ]:
wrong_class_2 = abs(class_df_2["Assigned Class"] - class_df_2["True Class"]).sum()/len(pima_df.index.values)
print(f"% of wrong classification: {100*wrong_class_2:.0f}%")

### 3c. Maximum Likelihood - Statistically Independent Features

In [ ]:
def classify_pima_3():
    
    class_df = pd.DataFrame(columns=["Assigned Class", "True Class"])
        
    for i in pima_df.index.values:
        temp_df = pima_df.drop(i)            
        temp_samples = dict([(key, [np.array(value[:-1]) for value in temp_df.values if value[-1]==key]) for key in pima_classes])
                
        mu_ml_0, covar_0 = ml_3(pd.DataFrame(temp_samples[0]))
        mu_ml_1, covar_1 = ml_3(pd.DataFrame(temp_samples[1]))

        x = np.array(pima_df.iloc[i, :-1])
        prob_0 = multivariate_normal.pdf(x=x, mean=mu_ml_0, cov=covar_0)
        prob_1 = multivariate_normal.pdf(x=x, mean=mu_ml_1, cov=covar_1)
        prob_class_0 = len(temp_samples[0])/(len(temp_samples[0]) + len(temp_samples[1]))
        prob_class_1 = len(temp_samples[1])/(len(temp_samples[0]) + len(temp_samples[1]))
        
        
        if prob_0*prob_class_0 > prob_1*prob_class_1:
            class_df.loc[i] = [0, pima_df.iloc[i, -1]]
        elif prob_1*prob_class_1 > prob_0*prob_class_0:
            class_df.loc[i] = [1, pima_df.iloc[i, -1]]
        else:
            class_df.loc[i] = ["NA", pima_df.iloc[i, -1]]
        
    return class_df

In [ ]:
class_df_3 = classify_pima_3()

In [ ]:
wrong_class_3 = abs(class_df_3["Assigned Class"] - class_df_3["True Class"]).sum()/len(pima_df.index.values)
print(f"% of wrong classification: {100*wrong_class_3:.0f}%")

### 3d. Bayesian Classification with use of Parzen Windows PDF Estimation

In [ ]:
def classify_pima_parzen():    
    
    class_df = pd.DataFrame(columns=["Assigned Class", "True Class"])
        
    for i in pima_df.index.values:
        temp_df = pima_df.drop(i)            
        temp_samples = dict([(key, [np.array(value[:-1]) for value in temp_df.values if value[-1]==key]) for key in pima_classes])

        x = np.array(pima_df.iloc[i, :-1])
        prob_0 = parzen_multivariate(np.sqrt(len(temp_df.index.values)), pd.DataFrame(temp_samples[0]).values, x)
        prob_1 = parzen_multivariate(np.sqrt(len(temp_df.index.values)), pd.DataFrame(temp_samples[1]).values, x)
        prob_class_0 = len(temp_samples[0])/(len(temp_samples[0]) + len(temp_samples[1]))
        prob_class_1 = len(temp_samples[1])/(len(temp_samples[0]) + len(temp_samples[1]))
        
        if prob_0*prob_class_0 > prob_1*prob_class_1:
            class_df.loc[i] = [0, pima_df.iloc[i, -1]]
        elif prob_1*prob_class_1 > prob_0*prob_class_0:
            class_df.loc[i] = [1, pima_df.iloc[i, -1]]
        else:
            class_df.loc[i] = ["NA", pima_df.iloc[i, -1]]
        
    return class_df

In [ ]:
class_df_parzen = classify_pima_parzen()

In [ ]:
wrong_class_parzen = abs(class_df_parzen["Assigned Class"] - class_df_parzen["True Class"]).sum()/len(pima_df.index.values)
print(f"% of wrong classification: {100*wrong_class_parzen:.0f}%")

## 4. Perceptron Classification

In [ ]:
def perceptron_classifier(class_1_df, class_2_df):
    
    class_1_df_copy = class_1_df.copy()
    class_2_df_copy = class_2_df.copy()
    
    class_1_df_copy["x_0"] = -1*np.ones(len(class_1_df_copy.index.values))
    class_2_df_copy["x_0"] = -1*np.ones(len(class_2_df_copy.index.values))
    class_1_df_copy["t"] = np.ones(len(class_1_df_copy.index.values))
    class_2_df_copy["t"] = -1*np.ones(len(class_2_df_copy.index.values))
    
    values_df = pd.concat((class_1_df_copy, class_2_df_copy), ignore_index=True)
    
    weights = np.zeros(len(values_df.columns[:-1].values))
    epoch_ok = 0
    while epoch_ok == 0:
        epoch_ok = 1
        for i in range(len(values_df)):
            t_x = np.array(values_df.iloc[i, values_df.columns.get_loc("t")]*values_df.iloc[i, :values_df.columns.get_loc("t")])
            t_w_x = t_x.reshape(len(values_df.columns[:-1].values), 1).transpose() @ weights.reshape(len(values_df.columns[:-1].values), 1)
            
            print(f"Iteration no. {i}")
            if t_w_x <= 0:
                epoch_ok = 0
                weights += t_x
            else:
                return weights

In [ ]:
def iris_perceptron():
    
    weights_dict = {}
    
    for iris_class in iris_classes:
        print(f"Isolating class {iris_class}")
        class_2_values = [value[i] for key, value in iris_class_values.items() if key not in [iris_class] for i in range(len(value))]
        class_2_df = pd.DataFrame(class_2_values)
        class_1_df = pd.DataFrame(iris_class_values[iris_class])
        
        weights_dict[iris_class] = perceptron_classifier(class_1_df, class_2_df)
    
    return weights_dict

In [ ]:
iris_perceptron_dict = iris_perceptron()

In [ ]:
print(iris_perceptron_dict)